loading required libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 5000)
import json
import re

importing the dataset

In [ ]:
filePath="dataset\clinicallab\data_en.json"
with open(filePath, 'r', encoding='utf-8') as f:
            data = json.load(f)
print("number of total cases are",len(data))
print("each case have the following fields",list(data[0].keys()))

filtering the dataset

In [ ]:

keys_to_include = ['clinical_department', 'principal_diagnosis', 'preliminary_diagnosis', 'diagnostic_basis', 'differential_diagnosis', 'treatment_plan', 'clinical_case_summary', 'imageological_examination', 'laboratory_examination', 'pathological_examination', 'therapeutic_principle']
df = pd.DataFrame([{key: d[key] for key in keys_to_include} for d in data])

In [ ]:
allDepartments=df['clinical_department'].value_counts()
print("number of departments available are",len(allDepartments))

print(allDepartments)

In [ ]:
def filterDepartment(df,department):
    allDepartments=df["clinical_department"].unique().tolist()
    if department in allDepartments:
        departmentdf=df[df["clinical_department"]==department]
    else:
        raise Exception
    return departmentdf

def extractList(s):
    pattern = r'\d+\.\s*'
    conditions = re.split(pattern, s)
    
    # If there's only one condition, it means no pattern was matched
    if len(conditions) == 1:
        return [s.strip()]
    
    # Remove any empty string that might result from the split
    return [condition.strip() for condition in conditions if condition.strip()]

def extract_information(text):
    sections = [
        "Patient Basic Information",
        "Chief Complaint",
        "Medical History",
        "Physical Examination",
        "Auxiliary Examination",
        "Imaging Examination",
        "Laboratory Examination",
        "Pathological Examination"
    ]
    
    # Create a regex pattern to match each section
    pattern = r'(' + '|'.join(re.escape(section) for section in sections) + r'):\s*'
    
    # Split the text based on the pattern
    matches = re.split(pattern, text)
    
    # The first element is the text before the first match (Case Summary), so we skip it
    matches = matches[1:]
    
    # Create the dictionary by pairing section names with their content
    result = {}
    for i in range(0, len(matches), 2):
        section = matches[i].strip()
        content = matches[i+1].strip()
        result[section] = content
    
    return result


In [ ]:

df['preliminary_diagnosis'] = df['preliminary_diagnosis'].apply(extractList)
df['diagnostic_basis'] = df['diagnostic_basis'].apply(extractList)
df['differential_diagnosis'] = df['differential_diagnosis'].apply(extractList)
df['treatment_plan'] = df['treatment_plan'].apply(extractList)
df["clinical_case_summary"] = df["clinical_case_summary"].apply(extract_information)

In [ ]:
def getDepartmentStatistics(df):
    prinicipal_diagnosis=df["principal_diagnosis"].value_counts()
    print("number of principal diagnosis are",len(prinicipal_diagnosis))
    print(prinicipal_diagnosis)
    preliminary_diagnosis=df["preliminary_diagnosis"].value_counts()
    print("number of preliminary_diagnosis are",len(prinicipal_diagnosis))
    print(preliminary_diagnosis)

In [ ]:
departmentdf=filterDepartment(df,"neurosurgery department")

In [ ]:
getDepartmentStatistics(departmentdf)

In [ ]:
from llm import doctor_prompt
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

In [ ]:
client = OpenAI()

In [ ]:
caseNumber=20#68
doctor="cardiologist"
row= departmentdf.iloc[caseNumber][['principal_diagnosis',"clinical_case_summary",'differential_diagnosis']]
medicalHistory=row.clinical_case_summary
principalDiagnosis=row.principal_diagnosis
differentialDiagnosis=row.differential_diagnosis

In [ ]:
print("the principal diagnosis is",principalDiagnosis)
print("differential diagnosis is",differentialDiagnosis)
doctor_prompt(client,medicalHistory,doctor)